# 02 · Codificación offline (opcional)

In [ ]:
from pathlib import Path; import h5py, cv2, numpy as np, pandas as pd
ROOT=Path.cwd().parent; PROC=ROOT/'data'/'processed'; RUN='circuito1'; SPLIT='train'
CSV=PROC/RUN/f'{SPLIT}.csv'; OUT=PROC/RUN/f'{SPLIT}_rate_T20_gain0.5.h5'; T=20; GAIN=0.5; SIZE=(160,80)
def rate_np(gray01,T,g): H,W=gray01.shape; rand=np.random.rand(T,H,W); return (rand<(gray01[None,:,:]*g)).astype(np.uint8)
df=pd.read_csv(CSV)
with h5py.File(OUT,'w') as h5:
    grp=h5.create_group('samples'); h5.create_dataset('steering', data=df['steering'].to_numpy(dtype=np.float32))
    base=ROOT/'data'/'raw'/'udacity'/RUN
    for idx,row in df.iterrows():
        bgr=cv2.imread(str((base/row['center']).resolve())); 
        if bgr is None: continue
        gray=cv2.cvtColor(bgr,cv2.COLOR_BGR2GRAY); gray=cv2.resize(gray,SIZE,interpolation=cv2.INTER_AREA)
        spk=rate_np(gray.astype(np.float32)/255.0, T, GAIN); grp.create_dataset(f'{idx}', data=spk, compression='gzip')
    h5.attrs['T']=T; h5.attrs['gain']=GAIN; h5.attrs['size']=SIZE
print('Guardado:', OUT)